In [4]:
from datasets import load_dataset

ds = load_dataset("nbroad/apigen-with-thinking-1.5k")['train']
ds

Dataset({
    features: ['messages', 'tools'],
    num_rows: 1500
})

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-14B")

In [8]:
import json


def tokenize(sample):
    full_ids = tokenizer.apply_chat_template(
        json.loads(sample["messages"]), tokenize=True, tools=json.loads(sample["tools"])
    )
    prefix_ids = tokenizer.apply_chat_template(
        json.loads(sample["messages"])[:-1], tokenize=True, tools=json.loads(sample["tools"]), add_generation_prompt=True
    )

    # Masking all tokens from loss except for the last assistant message
    labels = [-100] * len(prefix_ids) + full_ids[len(prefix_ids):]

    assert len(full_ids) == len(labels)


    return {
        "input_ids": full_ids,
        "attention_mask": [1] * len(full_ids),
        "labels": labels
    }


tokenized_ds = ds.map(tokenize, batched=False, num_proc=16)

Map (num_proc=16):   0%|          | 0/1500 [00:00<?, ? examples/s]

In [9]:
def visualize_masking(input_ids, labels):
    """
    Visualize token masking using ANSI colors.
    
    Args:
        input_ids: List of token IDs
        labels: List of labels (-100 for masked, token_id for unmasked)
        tokenizer: Optional tokenizer to convert IDs to tokens
        tokens: Optional list of token strings (if tokenizer not provided)
    """
    # ANSI color codes
    RED = '\033[31m'      # Masked tokens
    GREEN = '\033[32m'    # Unmasked tokens
    RESET = '\033[0m'     # Reset color
    BOLD = '\033[1m'      # Bold text

    
    print(f"\n{BOLD}Token Masking Visualization{RESET}")
    print(f"{RED}Red = Masked (-100){RESET}, {GREEN}Green = Unmasked (tokens to train on){RESET}")
    print("-" * 80)
    

    line_tokens = []
    line_length = 0

    tokens = [tokenizer.decode(x) for x in input_ids]
    
    for token, label in zip(tokens, labels):
        # Choose color based on masking
        if label == -100:
            colored_token = f"{RED}{token}{RESET}"
        else:
            colored_token = f"{GREEN}{token}{RESET}"
        

        token_info = f"{colored_token}"

        line_tokens.append(token_info)
        line_length += len(token)
    

    if line_tokens:
        print(''.join(line_tokens))

In [10]:
x = tokenized_ds.shuffle()[0]

visualize_masking(x["input_ids"], x["labels"])


Token Masking Visualization
Red = Masked (-100), Green = Unmasked (tokens to train on)
--------------------------------------------------------------------------------
<|im_start|>system
# Retail agent policy
As a retail agent, you can help users cancel or modify pending orders, return or exchange delivered orders, modify their default user address, or provide information about their own profile, orders, and related products.
- At the beginning of the conversation, you have to authenticate the user identity by locating their user id via email, or via name + zip code. This has to be done even when the user already provides the user id.
- Once the user has been authenticated, you can provide the user with information about order, product, profile information, e.g. help the user look up order id.
- You can only help one user per conversation (but you can handle multiple requests from the same user), and must deny any requests for tasks related to any other user.
- Before taking consequen